In [ ]:
# -----********************-----

# Created Time: 2025/07/13

# Last updated: 2025/07/21

# Author: Yiyi He

### Use Case

# This notebook explores the application of Granger Causality and autoregressive models
# 1.

# -----********************-----

# Libraries

In [ ]:
# Import libraries
import os
import warnings
from pathlib import Path
warnings.filterwarnings("ignore")

# Stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.api import ARDL
import statsmodels.api as sm
import numpy as np
from statsmodels.tsa.ardl import ardl_select_order
from statsmodels.tsa.stattools import grangercausalitytests

# Geo
from shapely.geometry import Point, Polygon
# import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 1000

# Plot
import matplotlib.pyplot as plt
# import seaborn as sns

# Processing
from tqdm import tqdm
import functools as ft
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

# Stationarity Test

### ADF

#### Hourly

In [36]:
# Hourly
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro

hourly_df = pd.read_csv(home_dir + "01_data/processed/csv/hourly_519station_3weather.csv",
                        index_col=0)

# Generate a list of station ids
station_lst = list(set(hourly_df.station_id.unique()))

adf_res = {}
for s_id in tqdm(station_lst):
        # Extracting the dataframe for a/one station
        station_df = hourly_df[hourly_df['station_id'] == s_id].sort_values(by='datetime')
        # Extact time series
        pct_blackout_ts = station_df.pct_blackout.values # Target
        t2m_ts = station_df.t2m.values # Predictor
        tp_ts = station_df.tp.values # Predictor
        wind_speed_ts = station_df.wind_speed.values # Predictor

        adf_res[s_id] = [
                        adfuller(pct_blackout_ts)[1], # p-value
                        adfuller(t2m_ts)[1],          # p-value
                        adfuller(tp_ts)[1],           # p-value
                        adfuller(wind_speed_ts)[1]    # p-value
        ]
adf_res_df = pd.DataFrame.from_dict(adf_res, orient='index').reset_index()
adf_res_df.rename(columns={
    'index': 'station_id',
    0: 'pct_blackout_pvalue',
    1: 't2m_pvalue',
    2: 'tp_pvalue',
    3: 'wind_speed_pvalue'
    }, inplace=True)
adf_res_df.to_csv(home_dir + '01_data/processed/csv/ADF/hourly_ADF_test_pvalues.csv')

100%|██████████| 519/519 [1:12:38<00:00,  8.40s/it]


In [171]:
# Summary statistics
hourly_adf_res = pd.read_csv(home_dir + '01_data/processed/csv/Stationarity_test/hourly_ADF_test_pvalues.csv',
                             index_col=0)

# summarize number of stationary time series
# (hourly_adf_res.iloc[:, 1:]<0.05).sum()
# Summary statistics for p values
(hourly_adf_res.iloc[:, 1:]).describe().iloc[1:].applymap(lambda x: f"{x:.3f}")

,pct_blackout_pvalue,t2m_pvalue,tp_pvalue,wind_speed_pvalue
mean,0.001,0.229,0.006,0.007
std,0.007,0.263,0.063,0.043
min,0.000,0.000,0.000,0.000
25%,0.000,0.010,0.000,0.000
50%,0.000,0.130,0.000,0.000
75%,0.000,0.358,0.000,0.000
max,0.140,0.951,0.903,0.571


#### Daily

In [133]:
# Daily
# Set home directory
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro

# Load station daily data
daily_df = pd.read_csv(home_dir + "01_data/processed/csv/daily_519station_13weather.csv", index_col=0)

# Generate a list of station ids
station_lst = list(set(daily_df.station_id.unique()))

# Predictors
weather_variables = ['t2m', 'wind_speed', 'tp']
aggregations = ['median', 'mean', 'max']
predictor_lst = ['daily_blackout_minutes'] + [w + "_" + a for w in weather_variables for a in aggregations]

adf_res = {}
for s_id in tqdm(station_lst):
        # Extracting the dataframe for a/one station
        station_df = daily_df[daily_df['station_id'] == s_id].sort_values(by='date')
        adf_res_lst = []
        for var in predictor_lst:
                # Extact time series
                ts = station_df[var].values
                adf_res_lst.append(adfuller(ts)[1]) # append pvalue

        adf_res[s_id] = adf_res_lst

adf_res_df = pd.DataFrame.from_dict(adf_res, orient='index').reset_index()
# Dictionary to update column names
rename_dict = {'index': 'station_id'}
rename_dict.update({i: var_name + '_pvalue' for i, var_name in enumerate(predictor_lst)})
# Rename columns
adf_res_df.rename(columns=rename_dict, inplace=True)
adf_res_df.to_csv(home_dir + '01_data/processed/csv/ADF/daily_ADF_test_pvalues.csv')

100%|██████████| 519/519 [05:09<00:00,  1.67it/s]


In [174]:
# Summary statistics
daily_adf_res = pd.read_csv(home_dir + '01_data/processed/csv/Stationarity_test/daily_ADF_test_pvalues.csv',
                             index_col=0)

# summarize number of stationary time series
(daily_adf_res.iloc[:, 1:]<0.05).sum()

# Summary statistics for p values
(daily_adf_res.iloc[:, 1:]).describe().iloc[1:].applymap(lambda x: f"{x:.3f}")

,daily_blackout_minutes_pvalue,t2m_median_pvalue,t2m_mean_pvalue,t2m_max_pvalue,wind_speed_median_pvalue,wind_speed_mean_pvalue,wind_speed_max_pvalue,tp_median_pvalue,tp_mean_pvalue,tp_max_pvalue
mean,0.030,0.432,0.440,0.329,0.044,0.049,0.093,0.077,0.072,0.083
std,0.121,0.317,0.316,0.281,0.133,0.143,0.165,0.171,0.163,0.184
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.000,0.134,0.154,0.078,0.000,0.000,0.000,0.000,0.000,0.000
50%,0.000,0.410,0.424,0.261,0.000,0.000,0.006,0.002,0.001,0.003
75%,0.000,0.680,0.682,0.526,0.017,0.025,0.116,0.064,0.056,0.057
max,0.948,1.000,1.000,0.999,0.979,0.997,0.999,1.000,0.999,1.000


## KPSS

### Hourly

In [143]:
# Hourly
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro

hourly_df = pd.read_csv(home_dir + "01_data/processed/csv/hourly_519station_3weather.csv",
                        index_col=0)

# Generate a list of station ids
station_lst = list(set(hourly_df.station_id.unique()))

kpss_res = {}
for s_id in tqdm(station_lst):
        # Extracting the dataframe for a/one station
        station_df = hourly_df[hourly_df['station_id'] == s_id].sort_values(by='datetime')
        # Extact time series
        pct_blackout_ts = station_df.pct_blackout.values # Target
        t2m_ts = station_df.t2m.values # Predictor
        tp_ts = station_df.tp.values # Predictor
        wind_speed_ts = station_df.wind_speed.values # Predictor

        kpss_res[s_id] = [
                        kpss(pct_blackout_ts, regression="c", nlags="auto")[1], # p-value
                        kpss(t2m_ts, regression="c", nlags="auto")[1],          # p-value
                        kpss(tp_ts, regression="c", nlags="auto")[1],           # p-value
                        kpss(wind_speed_ts, regression="c", nlags="auto")[1]    # p-value
        ]
kpss_res_df = pd.DataFrame.from_dict(kpss_res, orient='index').reset_index()
kpss_res_df.rename(columns={
    'index': 'station_id',
    0: 'pct_blackout_pvalue',
    1: 't2m_pvalue',
    2: 'tp_pvalue',
    3: 'wind_speed_pvalue'
    }, inplace=True)
kpss_res_df.to_csv(home_dir + '01_data/processed/csv/Stationarity_test/hourly_KPSS_test_pvalues.csv')

100%|██████████| 519/519 [00:30<00:00, 17.03it/s]


### Daily

In [149]:
# Daily
# Set home directory
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro

# Load station daily data
daily_df = pd.read_csv(home_dir + "01_data/processed/csv/daily_519station_13weather.csv", index_col=0)

# Generate a list of station ids
station_lst = list(set(daily_df.station_id.unique()))

# Predictors
weather_variables = ['t2m', 'wind_speed', 'tp']
aggregations = ['median', 'mean', 'max']
predictor_lst = ['daily_blackout_minutes'] + [w + "_" + a for w in weather_variables for a in aggregations]

kpss_res = {}
for s_id in tqdm(station_lst):
        # Extracting the dataframe for a/one station
        station_df = daily_df[daily_df['station_id'] == s_id].sort_values(by='date')
        kpss_res_lst = []
        for var in predictor_lst:
                # Extact time series
                ts = station_df[var].values
                kpss_res_lst.append(kpss(ts, regression="c", nlags="auto")[1]) # append pvalue

        kpss_res[s_id] = kpss_res_lst
        # break
kpss_res_df = pd.DataFrame.from_dict(kpss_res, orient='index').reset_index()
# Dictionary to update column names
rename_dict = {'index': 'station_id'}
rename_dict.update({i: var_name + '_pvalue' for i, var_name in enumerate(predictor_lst)})
# Rename columns
kpss_res_df.rename(columns=rename_dict, inplace=True)
kpss_res_df.to_csv(home_dir + '01_data/processed/csv/Stationarity_test/daily_KPSS_test_pvalues.csv')

100%|██████████| 519/519 [00:01<00:00, 306.22it/s]


## Differencing Timeseries (based on ADF test results)

In [43]:
def difference_series(series, order=1):
    """
    Perform differencing on a pandas Series.
    
    Parameters:
    -----------
    series : pd.Series
        The time series to difference
    order : int
        Order of differencing (1 for first difference, 2 for second, etc.)
    
    Returns:
    --------
    pd.Series
        Differenced series
    """
    differenced = series.copy()
    
    for i in range(order):
        differenced = differenced.diff().dropna()
    
    return differenced

### Hourly differecing order=1 (that is not stationary)

In [175]:
# Hourly
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro

hourly_df = pd.read_csv(home_dir + "01_data/processed/csv/hourly_519station_3weather.csv",
                        index_col=0)

# Generate a list of station ids
station_lst = list(set(hourly_df.station_id.unique()))

adf_res_df = pd.read_csv(home_dir + '01_data/processed/csv/ADF/hourly_ADF_test_pvalues.csv', index_col=0)

var_names = ['pct_blackout', 't2m', 'tp', 'wind_speed']

for s_id in station_lst:
    # check ADF test result for station
    ADF_result = adf_res_df[adf_res_df.station_id == s_id].values[0][1:]<0.05 # array of four boolean values
    if ADF_result.sum() == 4: # all variables are stationary, no action needed
        continue
    else:
        print(f'I am working on station id = {s_id}')
        if ADF_result[0] == True: # The target timeseries is stationary, therefore we only need to drop the first observation
            var_to_process = [var for var, keep in zip(var_names, ADF_result<0.05) if keep]
            # original station df
            station_df = hourly_df[hourly_df['station_id'] == s_id].sort_values(by='datetime')

            station_pct_blackout_new = station_df.pct_blackout.values[1:] # new array of pct blackout values (eliminated the first entry)
            station_new = pd.DataFrame(station_pct_blackout_new, columns=['pct_blackout_shift1'])
            for var in var_to_process:
                var_diff = difference_series(station_df[var]) # differencing order=1
                if adfuller(var_diff)[1]>0.05: # is order 1 differencing does not work
                    print(s_id, var)
                else:
                    # add var_diff as another column with correct name
                    station_new[f'{var}_diff1'] = var_diff.values
        else: # the target timeseries is not stationary, we need to difference it first
            station_pct_blackout_diff = difference_series(station_df['pct_blackout']) # difference the pct blackout
            station_new = pd.DataFrame(station_pct_blackout_diff.values, columns=['pct_blackout_diff1'])
            if adfuller(station_pct_blackout_diff)[1]>0.05: # is order 1 differencing does not work
                print(s_id, 'diff1 does not work for this target variable')
            else:
                # process predictor variables
                var_to_process = [var for var, keep in zip(var_names, ADF_result<0.05) if keep][1:] # take away pct blackout
                for var in var_to_process:
                    var_diff = difference_series(station_df[var]) # differencing order=1
                    if adfuller(var_diff)[1]>0.05: # is order 1 differencing does not work
                        print(s_id, var)
                    else:
                    # add var_diff as another column with correct name
                        station_new[f'{var}_diff1'] = var_diff.values

            
    station_new.to_csv(home_dir + f"01_data/processed/csv/hourly_diff1/station_{s_id}_hourly_diff1.csv")

Check if the results of differencing works or not, in other words, if the time series are indeed stationary now

In [202]:
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro

# Check if all the time series in the differenced dataset are stationary
diff_dir = home_dir + "01_data/processed/csv/hourly_diff1/"
for item in os.listdir(diff_dir):
    if item[-3:] == 'csv':
        df = pd.read_csv(os.path.join(diff_dir, item), index_col=0)
        # Perform ADF test on all columns
        for col in df.columns:
            col_ts = df[col].values
            adf_res = adfuller(col_ts)[1]
            if adf_res >= 0.05:
                # Print filename and column name if time series is not stationary
                print(item, col) 

After differencing (order=1), all the time series are now stationary.

### Daily differecing order=1 (that is not stationary)

In [418]:
# Hourly
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro

daily_df = pd.read_csv(home_dir + "01_data/processed/csv/daily_519station_13weather.csv",
                        index_col=0)

# Generate a list of station ids
station_lst = list(set(daily_df.station_id.unique()))

adf_res_df = pd.read_csv(home_dir + '01_data/processed/csv/Stationarity_test/daily_ADF_test_pvalues.csv', index_col=0)

# Variable names
weather_variables = ['t2m', 'wind_speed', 'tp']
aggregations = ['median', 'mean', 'max']
var_names = ['daily_blackout_minutes'] + [w + "_" + a for w in weather_variables for a in aggregations]

for s_id in tqdm(station_lst):
    # check ADF test result for station
    ADF_result = adf_res_df[adf_res_df.station_id == s_id].values[0][1:]<0.05 # array of four boolean values indicating stationarity for variables

    if ADF_result.sum() == len(var_names): # all variables are stationary, no action needed
        continue
    else:
        # print(f'I am working on station id = {s_id}')
        if ADF_result[0] == True: # The target timeseries is stationary, therefore we only need to drop the first observation
            var_to_process = [var for var, keep in zip(var_names, ADF_result<0.05) if keep] # list of variables that are not stationary
            # original station df
            station_df = daily_df[daily_df['station_id'] == s_id].sort_values(by='date')
            
            station_blackout_minutes_new = station_df.daily_blackout_minutes.values[1:] # new array of pct blackout values (eliminated the first entry)
            station_new = pd.DataFrame(station_blackout_minutes_new, columns=['blackout_minutes_shift1'])

            flag = 1
            # First check if any var needs second order differencing
            for var in var_to_process:
                var_diff = difference_series(station_df[var]) # differencing order=1
                if adfuller(var_diff)[1]>=0.05:
                    flag = 2

            if flag == 1:
            # order 1 differencing is enough for this station
                for var in var_to_process:
                    var_diff = difference_series(station_df[var]) # differencing order=1
                    # add var_diff as another column with correct name
                    station_new[f'{var}_diff1'] = var_diff.values
            elif flag == 2:
                # need to shift blackout minutes by 2
                station_blackout_minutes_new = station_df.daily_blackout_minutes.values[2:] # new array of pct blackout values (eliminated the first 2 entries)
                station_new = pd.DataFrame(station_blackout_minutes_new, columns=['blackout_minutes_shift2']) # redefine station_new dataframe
                for var in var_to_process:
                    var_diff = difference_series(station_df[var]) # differencing order=1
                    # check if this variable needs second differencing
                    if adfuller(var_diff)[1]>=0.05:
                        var_diff2 = difference_series(var_diff) # second diff
                        station_new[f'{var}_diff2'] = var_diff2.values
                    else: # this variable does not need second differencing
                        var_diff = var_diff.values[1:]
                        station_new[f'{var}_diff1'] = var_diff

        else: # the target timeseries is not stationary, we need to difference it first
            station_blackout_minutes_diff = difference_series(station_df['daily_blackout_minutes']) # difference the daily blackout minutes
            station_new = pd.DataFrame(station_blackout_minutes_diff.values, columns=['daily_blackout_minutes_diff1'])
            if adfuller(station_blackout_minutes_diff)[1]>=0.05: # is order 1 differencing does not work for blackout minutes
                print(s_id, 'diff1 does not work for this target variable')
            else:
                # process predictor variables
                var_to_process = [var for var, keep in zip(var_names, ADF_result<0.05) if keep][1:] # take away blackout minutes

                flag = 1
                # First check if any var needs second order differencing
                for var in var_to_process:
                    var_diff = difference_series(station_df[var]) # differencing order=1
                    if adfuller(var_diff)[1]>=0.05:
                        flag = 2

                if flag == 1:
                # order 1 differencing is enough for this station
                    for var in var_to_process:
                        var_diff = difference_series(station_df[var]) # differencing order=1
                        # add var_diff as another column with correct name
                        station_new[f'{var}_diff1'] = var_diff.values

                elif flag == 2:
                    # need to shift blackout minutes by 2
                    station_blackout_minutes_new = station_df.daily_blackout_minutes.values[2:] # new array of pct blackout values (eliminated the first 2 entries)
                    station_new = pd.DataFrame(station_blackout_minutes_new, columns=['blackout_minutes_shift2']) # redefine station_new dataframe
                    for var in var_to_process:
                        var_diff = difference_series(station_df[var]) # differencing order=1
                        # check if this variable needs second differencing
                        if adfuller(var_diff)[1]>=0.05:
                            var_diff2 = difference_series(var_diff) # second diff
                            station_new[f'{var}_diff2'] = var_diff2.values
                        else: # this variable does not need second differencing
                            var_diff = var_diff.values[1:]
                            station_new[f'{var}_diff1'] = var_diff

    station_new.to_csv(home_dir + f"01_data/processed/csv/daily_diff1/station_{s_id}_daily_diff{flag}.csv")
    # break

100%|██████████| 519/519 [01:09<00:00,  7.45it/s]


Check if the differencing works for all stations and all variables.

In [420]:
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro

# Check if all the time series in the differenced dataset are stationary
diff_dir = home_dir + "01_data/processed/csv/daily_diff1/"
for item in os.listdir(diff_dir):
    if item[-3:] == 'csv':
        df = pd.read_csv(os.path.join(diff_dir, item), index_col=0)
        # Perform ADF test on all columns
        for col in df.columns:
            col_ts = df[col].values
            adf_res = adfuller(col_ts)[1]
            if adf_res >= 0.05:
                # Print filename and column name if time series is not stationary
                print(item, col)

After stationarity check, the following six stations will be removed from the daily analysis:
- station id: 183, 423, 559, 315, 103, 62

# Granger Causality

## GC with hourly data

In [25]:
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/'

hourly_df = pd.read_csv(home_dir + "01_data/processed/csv/hourly_519station_3weather.csv",
                        index_col=0)

Define function that runs the Granger Causality test on hourly station data. \
For each station, run the Granger Causality test with the following set up:\
 \
**Target variable**: \
'pct_blackout' \
**Predictor variables**: 
1. 't2m' (temperature) 
2. 'tp' (precipitation) 
3. 'wind_speed' (wind speed)

In [29]:
def run_gc_hourly(maxlag, target, predictor, station_lst, hourly_df):
    # Initiate an empty dictionary for storing test results
    gc_dic = {}

    # Iterate through all stations
    for s_id in tqdm(station_lst):
        # Extracting the dataframe for a/one station
        station_df = hourly_df[hourly_df['station_id'] == s_id].sort_values(by='datetime')
        try:
            # Check if the number of observations available at the station is sufficient for the gc model
            if len(station_df) <= maxlag + 2 + 1: # maxlag + num_variables (target, predictor) + 1
                print(f"Skipping Station {s_id} due to insufficient observations for maxlag={maxlag}")
                continue
            else:
                # Run granger causality test on station hourly data
                test_result = grangercausalitytests(
                    station_df[[target, predictor]], maxlag=maxlag, addconst=True, verbose=False)
                # Save test values
                F_test_p_values = [round(test_result[i+1][0]['ssr_ftest'][1],4) for i in range(maxlag)]
                Chi_squared_p_values = [round(test_result[i+1][0]['ssr_chi2test'][1],4) for i in range(maxlag)]
                p_values_min = np.min(F_test_p_values+Chi_squared_p_values)
                # Key: station id, Value: list of 1. minimum F/Chi p values 2. F-test p values for all lags 3. Chi-square test p-values for all lags
                gc_dic[s_id] = [p_values_min, F_test_p_values, Chi_squared_p_values]
        except ValueError:
            print(f"Skipping Station {s_id} due to insufficient observations for maxlag={maxlag}")
    # Convert to dataframe        
    gc_df = pd.DataFrame.from_dict(gc_dic, orient='index').reset_index()
    gc_df.rename(columns={
    'index':'station_id',
    0:f'{predictor}_p-value_min',
    1:f'{predictor}_f_p-value',
    2:f'{predictor}_Chi_p-value'
    }, inplace=True)
    return gc_df

In [ ]:
# Set home directory
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/'
# Read hourly data
hourly_df = pd.read_csv(home_dir + "01_data/processed/csv/hourly_519station_3weather.csv",
                        index_col=0)
# Set target variable
target = 'pct_blackout'
# Set max lag for all variables
maxlag = 3
# Generate a list of station ids
station_lst = list(set(hourly_df.station_id.unique()))
# List of predictor variables
predictor_lst = ['t2m', 'tp', 'wind_speed']

# Initiate empty dictionary to store test output dataframes
res_dic = {}
# Loop through the list of predictors
for predictor in predictor_lst:
    # Run gc test
    df_res = run_gc_hourly(maxlag, target, predictor, station_lst, hourly_df)
    # Save output dataframe in dictionary key: predictor, value: dataframe with test values
    res_dic[predictor] = df_res

# Join resulting dataframes for all predictors together
dfs = [res_dic[p] for p in predictor_lst]
df_joined= ft.reduce(lambda left, right: pd.merge(left, right, on='station_id'), dfs)
# Save output as new csv file
df_joined.to_csv(home_dir + f"01_data/processed/csv/GC/granger_hourly_max{maxlag}_pvalue.csv")

100%|██████████| 519/519 [00:23<00:00, 22.22it/s]


### GC on hourly data after first-order differencing


In [33]:
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
# home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro
# Folder that contains the differenced data
diff_dir = "01_data/processed/csv/hourly_diff1"
# Set max lag
maxlag = 3
# Iterate through station files (first order differencing applied and all time series are stationary)
for file in tqdm(os.listdir(home_dir + diff_dir)):
    if file[-3:] == 'csv':
        # Extract station id
        station_id = int(file.split('_')[1])
        df = pd.read_csv(os.path.join(home_dir + diff_dir, file), index_col=0)
        # Initiate empty dictionary to store results
        gc_res = {}
        # Target time series: percent blackout
        target_ts = df.iloc[:, 0].values
        # Go though rest of the columns and perform GC test. Note, stations have different column numbers, depending on how many of the original are not stationary
        try:
            # Check if the number of observations available at the station is sufficient for the gc model
            if len(df) <= maxlag + 2 + 1: # maxlag + num_variables (target, predictor) + 1
                print(f"Skipping Station {station_id} due to insufficient observations for maxlag={maxlag}")
                continue
            else:
                for col in list(df.columns)[1:]:
                    col_name = col.split('_')[0]
                    test_result = grangercausalitytests(
                                                        df[[list(df.columns)[0], col]],
                                                        maxlag=maxlag,
                                                        addconst=True,
                                                        verbose=False)
                    F_test_p_values = [round(test_result[i+1][0]['ssr_ftest'][1],4) for i in range(maxlag)]
                    Chi_squared_p_values = [round(test_result[i+1][0]['ssr_chi2test'][1],4) for i in range(maxlag)]
                    p_values_min = np.min(F_test_p_values+Chi_squared_p_values)
                    gc_res[col + '_p-value_min'] = p_values_min
                    gc_res[col + '_f_p-value'] = F_test_p_values
                    gc_res[col + '_Chi_p-value'] = Chi_squared_p_values
                # Store results in a dataframe
                gc_res_df = pd.DataFrame.from_dict(gc_res)
                # Save station results to new csv
                gc_res_df.to_csv(home_dir + f"01_data/processed/csv/GC/hourly_station_diff1/hourly_station_maxlag_{maxlag}/station_{station_id}.csv")
        except ValueError:
            print(f"Skipping Station {station_id} due to insufficient observations for maxlag={maxlag}")
    else:
        continue

100%|██████████| 329/329 [12:59<00:00,  2.37s/it]


### Update original GC hourly results

In [35]:
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
# home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro

maxlag = 3
diff_res_dir = f"01_data/processed/csv/GC/hourly_station_diff1/hourly_station_maxlag_{maxlag}"

# Load original gc result dataframe
original_gc_hourly = pd.read_csv(home_dir+f'01_data/processed/csv/GC/hourly/granger_hourly_max{maxlag}_pvalue.csv',
                                 index_col=0)

original_gc_hourly_stations = original_gc_hourly.station_id.values

for file in os.listdir(os.path.join(home_dir, diff_res_dir)):
    # Extract station id
    station_id = int(file.split('_')[1][:-4])
    if station_id in original_gc_hourly_stations:
        if file[-3:] == 'csv':
            # Read dataframe
            station_diff_gc = pd.read_csv(os.path.join(home_dir, diff_res_dir, file), index_col=0)
            # Now we are ready to extract the outputs from this dataframe
            var_list = list(set([i.split("_diff1")[0] for i in station_diff_gc.columns.values]))
            for var in var_list:
                var_pvalue_min = station_diff_gc[f'{var}_diff1_p-value_min'].values[0]
                var_f_pvalues = list(station_diff_gc[f'{var}_diff1_f_p-value'].values)
                var_chi_pvalues = list(station_diff_gc[f'{var}_diff1_Chi_p-value'].values)
                # Update original dataframe            
                row_idx  = original_gc_hourly[original_gc_hourly['station_id'] == station_id].index
                original_gc_hourly.at[row_idx.item(), f'{var}_p-value_min'] = var_pvalue_min.item()
                original_gc_hourly.at[row_idx.item(), f'{var}_f_p-value'] = [float(x) for x in var_f_pvalues]
                original_gc_hourly.at[row_idx.item(), f'{var}_Chi_p-value'] = [float(x) for x in var_chi_pvalues]
    else:
        continue
original_gc_hourly.to_csv(home_dir + f'01_data/processed/csv/GC/hourly/granger_hourly_max{maxlag}_pvalue_diff_update.csv')


In [210]:
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro

maxlag = 24 # hourly lags: 24, 72, 120
gc_hourly_original = pd.read_csv(home_dir + f"01_data/processed/csv/GC/granger_hourly_max{maxlag}_pvalue.csv",
                                 index_col=0)
gc_hourly_original.head(3)

,station_id,t2m_p-value_min,t2m_f_p-value,t2m_Chi_p-value,tp_p-value_min,tp_f_p-value,tp_Chi_p-value,wind_speed_p-value_min,wind_speed_f_p-value,wind_speed_Chi_p-value
0,1,0.0271,"[0.0272, 0.0746, 0.1287, 0.1963, 0.3036, 0.380...","[0.0271, 0.0742, 0.1279, 0.195, 0.3017, 0.378,...",0.0985,"[0.786, 0.1352, 0.0992, 0.1542, 0.2243, 0.2233...","[0.7859, 0.1347, 0.0985, 0.153, 0.2224, 0.221,...",0.2261,"[0.3995, 0.3685, 0.5548, 0.5856, 0.5533, 0.294...","[0.3993, 0.3679, 0.5539, 0.5844, 0.5515, 0.292..."
1,2,0.0000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0453,"[0.9078, 0.4611, 0.6229, 0.4407, 0.162, 0.0653...","[0.9078, 0.461, 0.6227, 0.4404, 0.1616, 0.065,...",0.0000,"[0.0004, 0.0005, 0.0003, 0.0007, 0.0014, 0.000...","[0.0004, 0.0004, 0.0003, 0.0007, 0.0014, 0.000..."
2,3,0.0000,"[0.0737, 0.4865, 0.2713, 0.0608, 0.0, 0.0, 0.0...","[0.0736, 0.4862, 0.2707, 0.0604, 0.0, 0.0, 0.0...",0.3917,"[0.8354, 0.9068, 0.644, 0.527, 0.55, 0.6509, 0...","[0.8353, 0.9067, 0.6436, 0.5262, 0.5489, 0.649...",0.0236,"[0.0628, 0.2473, 0.4359, 0.0495, 0.065, 0.1052...","[0.0627, 0.2469, 0.4353, 0.0491, 0.0644, 0.104..."


## GC with daily data

In [ ]:
def run_gc_daily(maxlag, target, predictor, station_lst, daily_df):
    # Initiate an empty dictionary for storing test results
    gc_dic = {}

    # Iterate through all stations
    for s_id in tqdm(station_lst):
        # Extracting the dataframe for a/one station
        station_df = daily_df[daily_df['station_id'] == s_id].sort_values(by='date') # Sort by date
        try:
            # Check if the number of observations available at the station is sufficient for the gc model
            if len(station_df) <= maxlag + 2 + 1: # maxlag + num_variables (target, predictor) + 1
                print(f"Skipping Station {s_id} due to insufficient observations for maxlag={maxlag}")
                continue
            else:
                # Run granger causality test on station daily data
                test_result = grangercausalitytests(
                    station_df[[target, predictor]], maxlag=maxlag, addconst=True, verbose=False)
                # Save test values
                F_test_p_values = [round(test_result[i+1][0]['ssr_ftest'][1],4) for i in range(maxlag)]
                Chi_squared_p_values = [round(test_result[i+1][0]['ssr_chi2test'][1],4) for i in range(maxlag)]
                p_values_min = np.min(F_test_p_values+Chi_squared_p_values)
                # Key: station id, Value: list of 1. minimum F/Chi p values 2. F-test p values for all lags 3. Chi-square test p-values for all lags
                gc_dic[s_id] = [p_values_min, F_test_p_values, Chi_squared_p_values]
        except ValueError:
            print(f"Skipping Station {s_id} due to insufficient observations for maxlag={maxlag}")
    # Convert to dataframe        
    gc_df = pd.DataFrame.from_dict(gc_dic, orient='index').reset_index()
    gc_df.rename(columns={
    'index':'station_id',
    0:f'{predictor}_p-value_min',
    1:f'{predictor}_f_p-value',
    2:f'{predictor}_Chi_p-value'
    }, inplace=True)
    return gc_df

In [355]:
# Set home directory
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/'
# Load station daily data
daily_df = pd.read_csv(home_dir + "01_data/processed/csv/daily_519station_13weather.csv", index_col=0)

# Target
target = "daily_blackout_minutes"
# Predictors
weather_variables = ['t2m', 'wind_speed', 'tp']
aggregations = ['median', 'mean', 'max']
predictor_lst = [w + "_" + a for w in weather_variables for a in aggregations]

# Create a list of unique station ids
station_id_lst = list(set(daily_df.station_id.unique()))

# Loop through multiple max lag values
for maxlag in [3, 5, 7, 15, 30, 45, 60]:
    # Initiate empty dictionary to store test output dataframes
    res_dic = {}
    # Loop through the list of predictors
    for predictor in predictor_lst:
        # Run gc test
        df_res = run_gc_daily(maxlag, target, predictor, station_lst, daily_df)
        # Save output dataframe in dictionary key: predictor, value: dataframe with test values
        res_dic[predictor] = df_res

    # Join resulting dataframes for all predictors together
    dfs = [res_dic[p] for p in predictor_lst]
    df_joined= ft.reduce(lambda left, right: pd.merge(left, right, on='station_id'), dfs)
    # Save output as new csv file
    df_joined.to_csv(home_dir + f"01_data/processed/csv/granger_daily_max{maxlag}_pvalue.csv")

### GC on daily data after first-order differencing

In [ ]:
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro
# Folder that contains the differenced data
diff_dir = "01_data/processed/csv/daily_diff1"
# Set max lag
maxlag = 60   # 3, 5, 7, 15, 30, 45, 60
# Iterate through station files (first order differencing applied and all time series are stationary)
for file in tqdm(os.listdir(home_dir + diff_dir)):
    if file[-3:] == 'csv':
        # Extract station id
        station_id = int(file.split('_')[1])
        df = pd.read_csv(os.path.join(home_dir + diff_dir, file), index_col=0)
        # Initiate empty dictionary to store results
        gc_res = {}
        # Target time series: percent blackout
        target_ts = df.iloc[:, 0].values
        # Go though rest of the columns and perform GC test. Note, stations have different column numbers, depending on how many of the original are not stationary
        try:
            # Check if the number of observations available at the station is sufficient for the gc model
            if len(df) <= maxlag + 2 + 1: # maxlag + num_variables (target, predictor) + 1
                print(f"Skipping Station {station_id} due to insufficient observations for maxlag={maxlag}")
                continue
            else:
                for col in list(df.columns)[1:]:
                    col_name = col.split('_diff')[0]
                    test_result = grangercausalitytests(
                                                        df[[list(df.columns)[0], col]],
                                                        maxlag=maxlag,
                                                        addconst=True,
                                                        verbose=False
                                                        )
                    F_test_p_values = [round(test_result[i+1][0]['ssr_ftest'][1],4) for i in range(maxlag)] # this collects the p values for all lags
                    Chi_squared_p_values = [round(test_result[i+1][0]['ssr_chi2test'][1],4) for i in range(maxlag)] # similar here
                    p_values_min = np.min(F_test_p_values+Chi_squared_p_values)
                    gc_res[col + '_p-value_min'] = p_values_min
                    gc_res[col + '_f_p-value'] = F_test_p_values
                    gc_res[col + '_Chi_p-value'] = Chi_squared_p_values
                # Store results in a dataframe
                gc_res_df = pd.DataFrame.from_dict(gc_res)
                # Save station results to new csv
                gc_res_df.to_csv(home_dir + f"01_data/processed/csv/GC/daily_station_diff1/daily_station_maxlag_{maxlag}/station_{station_id}.csv")
        except ValueError:
            print(f"Skipping Station {station_id} due to insufficient observations for maxlag={maxlag}")
    else:
        continue

### Update original GC daily results

In [486]:
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro

maxlag = 60 # 3, 5, 7, 15, 30, 45, 60
diff_res_dir = f"01_data/processed/csv/GC/daily_station_diff1/daily_station_maxlag_{maxlag}"

# Load original gc result dataframe
original_gc_daily = pd.read_csv(home_dir+f'01_data/processed/csv/GC/daily/granger_daily_max{maxlag}_pvalue.csv',
                                 index_col=0)

original_gc_daily_stations = original_gc_daily.station_id.values

# Iterate through files in differenced daily time series folder
for file in tqdm(os.listdir(os.path.join(home_dir, diff_res_dir))):
    # Extract station id
    station_id = int(file.split('_')[1][:-4])
    if station_id in original_gc_daily_stations:
        if file[-3:] == 'csv':
            # Read dataframe
            station_diff_gc = pd.read_csv(os.path.join(home_dir, diff_res_dir, file), index_col=0)
            # Now we are ready to extract the outputs from this dataframe
            var_list = list(set([i.split("_diff")[0] for i in station_diff_gc.columns.values]))
            for var in var_list:
                var_pvalue_min = station_diff_gc.filter(regex=rf'{var}_diff\d_p-value_min').values.flatten()[0]
                var_f_pvalues = list(station_diff_gc.filter(regex=rf'{var}_diff\d_f_p-value').values.flatten())
                var_chi_pvalues = list(station_diff_gc.filter(regex=rf'{var}_diff\d_Chi_p-value').values.flatten())
                # Update original dataframe            
                row_idx  = original_gc_daily[original_gc_daily['station_id'] == station_id].index
                original_gc_daily.at[row_idx.item(), f'{var}_p-value_min'] = var_pvalue_min.item()
                original_gc_daily.at[row_idx.item(), f'{var}_f_p-value'] = [float(x) for x in var_f_pvalues]
                original_gc_daily.at[row_idx.item(), f'{var}_Chi_p-value'] = [float(x) for x in var_chi_pvalues]
    else:
        continue
original_gc_daily.to_csv(home_dir + f'01_data/processed/csv/GC/daily/granger_daily_max{maxlag}_pvalue_diff_update.csv')


100%|██████████| 391/391 [00:02<00:00, 141.66it/s]


## Summarize GC results

### Hourly

In [71]:
# for each maxlag clean the gc results dataframe
# select only min p-values
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac

maxlags = [3, 24, 72, 120]

var_lst = ['t2m', 'tp', 'wind_speed']
cols_select = [var+'_p-value_min' for var in var_lst]

for maxlag in maxlags:
    # Hourly gc results (updated with differenced time series)
    hourly_gc_res = pd.read_csv(home_dir + f'01_data/processed/csv/GC/hourly/granger_hourly_max{maxlag}_pvalue_diff_update.csv',
                                index_col=0)
    hourly_gc_select = hourly_gc_res[['station_id'] + cols_select]
    hourly_gc_select.to_csv(home_dir+f'01_data/processed/csv/GC/hourly/granger_hourly_max{maxlag}_pvalue_diff_update_select.csv')

In [64]:
maxlags = [3, 24, 72, 120]
var_lst = ['t2m', 'tp', 'wind_speed']

for maxlag in maxlags:
    gc_hourly_select_df = pd.read_csv(home_dir+f'01_data/processed/csv/GC/hourly/granger_hourly_max{maxlag}_pvalue_diff_update_select.csv',
                                    index_col=0)
    number_of_stations = len(set(gc_hourly_select_df.station_id.values))
    print(f"Summarizing GC results for maxlag={maxlag}")
    print(f"Number of stations included in the analysis: {number_of_stations}")
    for var in var_lst:
        x = sum((gc_hourly_select_df[f'{var}_p-value_min']<0.05)*1)
        print(f"{x} out of {number_of_stations} stations, {var} Granger Cause blackout percent.")
    print(".....................")

Summarizing GC results for maxlag=3
Number of stations included in the analysis: 519
412 out of 519 stations, t2m Granger Cause blackout percent.
216 out of 519 stations, tp Granger Cause blackout percent.
300 out of 519 stations, wind_speed Granger Cause blackout percent.
.....................
Summarizing GC results for maxlag=24
Number of stations included in the analysis: 517
485 out of 517 stations, t2m Granger Cause blackout percent.
406 out of 517 stations, tp Granger Cause blackout percent.
440 out of 517 stations, wind_speed Granger Cause blackout percent.
.....................
Summarizing GC results for maxlag=72
Number of stations included in the analysis: 517
498 out of 517 stations, t2m Granger Cause blackout percent.
438 out of 517 stations, tp Granger Cause blackout percent.
465 out of 517 stations, wind_speed Granger Cause blackout percent.
.....................
Summarizing GC results for maxlag=120
Number of stations included in the analysis: 512
499 out of 512 stations

### Daily

In [72]:
# for each maxlag clean the gc results dataframe
# select only min p-values
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac

maxlags = [3, 5, 7, 15, 30, 45, 60]

var_lst = ['t2m_max', 'tp_max', 'wind_speed_max']
cols_select = [var+'_p-value_min' for var in var_lst]

for maxlag in maxlags:
    # Hourly gc results (updated with differenced time series)
    daily_gc_res = pd.read_csv(home_dir + f'01_data/processed/csv/GC/daily/granger_daily_max{maxlag}_pvalue_diff_update.csv',
                                index_col=0)
    daily_gc_select = daily_gc_res[['station_id'] + cols_select]
    daily_gc_select.to_csv(home_dir+f'01_data/processed/csv/GC/daily/granger_daily_max{maxlag}_pvalue_diff_update_select.csv')

In [74]:
maxlags = [5, 15, 30, 45, 60]
var_lst = ['t2m_max', 'tp_max', 'wind_speed_max']

for maxlag in maxlags:
    gc_daily_select_df = pd.read_csv(home_dir+f'01_data/processed/csv/GC/daily/granger_daily_max{maxlag}_pvalue_diff_update_select.csv',
                                    index_col=0)
    number_of_stations = len(set(gc_daily_select_df.station_id.values))
    print(f"Summarizing GC results for maxlag={maxlag}")
    print(f"Number of stations included in the analysis: {number_of_stations}")
    for var in var_lst:
        x = sum((gc_daily_select_df[f'{var}_p-value_min']<0.05)*1)
        print(f"{x} out of {number_of_stations} stations, {var} Granger Cause blackout percent.")
    print(".....................")

Summarizing GC results for maxlag=5
Number of stations included in the analysis: 512
132 out of 512 stations, t2m_max Granger Cause blackout percent.
196 out of 512 stations, tp_max Granger Cause blackout percent.
111 out of 512 stations, wind_speed_max Granger Cause blackout percent.
.....................
Summarizing GC results for maxlag=15
Number of stations included in the analysis: 500
201 out of 500 stations, t2m_max Granger Cause blackout percent.
250 out of 500 stations, tp_max Granger Cause blackout percent.
177 out of 500 stations, wind_speed_max Granger Cause blackout percent.
.....................
Summarizing GC results for maxlag=30
Number of stations included in the analysis: 472
255 out of 472 stations, t2m_max Granger Cause blackout percent.
290 out of 472 stations, tp_max Granger Cause blackout percent.
246 out of 472 stations, wind_speed_max Granger Cause blackout percent.
.....................
Summarizing GC results for maxlag=45
Number of stations included in the an

# ARDL

Correlation between daily weather aggregate variables

In [4]:
# Set home directory
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/'
# Load station daily data
daily_df = pd.read_csv(home_dir + "01_data/processed/csv/daily_519station_13weather.csv", index_col=0)

# Correlation
correlation_matrix = daily_df[['daily_blackout_minutes',
                                        't2m_min',
                                        't2m_median',
                                        't2m_max',
                                        'wind_speed_median',
                                        'wind_speed_max',
                                        'tp_median',
                                        'tp_max']].corr()
correlation_matrix.style.background_gradient(cmap='coolwarm')

,daily_blackout_minutes,t2m_min,t2m_median,t2m_max,wind_speed_median,wind_speed_max,tp_median,tp_max
daily_blackout_minutes,1.000000,0.020774,0.027144,0.024727,-0.002525,0.004347,0.034892,0.040650
t2m_min,0.020774,1.000000,0.911219,0.725396,0.265783,0.357374,0.192918,0.218318
t2m_median,0.027144,0.911219,1.000000,0.921124,0.187497,0.300769,0.007608,0.023848
t2m_max,0.024727,0.725396,0.921124,1.000000,0.102492,0.235172,-0.170036,-0.169363
wind_speed_median,-0.002525,0.265783,0.187497,0.102492,1.000000,0.872144,0.091777,0.104201
wind_speed_max,0.004347,0.357374,0.300769,0.235172,0.872144,1.000000,0.067437,0.087814
tp_median,0.034892,0.192918,0.007608,-0.170036,0.091777,0.067437,1.000000,0.812423
tp_max,0.040650,0.218318,0.023848,-0.169363,0.104201,0.087814,0.812423,1.000000


Given that the aggregate of one particular weather variable is highly correlated with one another. We decide to use the maximum for each weather variable: t2m_max, tp_max, and wind_speed_max

In [5]:
correlation_matrix = daily_df[['daily_blackout_minutes',
                                        't2m_max',
                                        'wind_speed_max',
                                        'tp_max']].corr()
correlation_matrix.style.background_gradient(cmap='coolwarm')

,daily_blackout_minutes,t2m_max,wind_speed_max,tp_max
daily_blackout_minutes,1.000000,0.024727,0.004347,0.040650
t2m_max,0.024727,1.000000,0.235172,-0.169363
wind_speed_max,0.004347,0.235172,1.000000,0.087814
tp_max,0.040650,-0.169363,0.087814,1.000000


The results show low correlation between the max of three weather variables.

## Hourly

In [ ]:
def extract_ardl_results(res, station_id, sel_res=None):
    """
    Extract key results from fitted ARDL model
    """
    # Basic model info
    results_dict = {
        'station_id': station_id,
        'model_type': 'ARDL',
        'endog_var': res.model.endog_names,
        'exog_var': res.model.exog_names,
        'n_obs': res.nobs,
        'al_lags': str(sel_res.model.ar_lags),
        'dl_lags': str(sel_res.model.dl_lags),
        'log_likelihood': res.llf,
        'aic': res.aic,
        'bic': res.bic,
        'hqic': res.hqic,
        'fitted_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # Extract coefficients, std errors, t-stats, p-values
    params = res.params
    std_errors = res.bse
    tvalues = res.tvalues
    pvalues = res.pvalues
    conf_int = res.conf_int()
    
    # Store coefficient information
    for i, param_name in enumerate(params.index):
        results_dict[f'coef_{param_name}'] = params.iloc[i]
        results_dict[f'se_{param_name}'] = std_errors.iloc[i]
        results_dict[f'tstat_{param_name}'] = tvalues.iloc[i]
        results_dict[f'pvalue_{param_name}'] = pvalues.iloc[i]
        results_dict[f'ci_lower_{param_name}'] = conf_int.iloc[i, 0]
        results_dict[f'ci_upper_{param_name}'] = conf_int.iloc[i, 1]
    
    return results_dict

In [ ]:
# Set home directory
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac

# Original hourly dataset (without differencing) is stored
hourly_df = pd.read_csv(home_dir + "01_data/processed/csv/hourly_519station_3weather.csv",
                        index_col=0)

# Station ids where there EXISTS a non-stationary time series in the station data
hourly_diff_station_dir = "01_data/processed/csv/diff/hourly_diff1"
hourly_diff_station_ids = []
for file in os.listdir(os.path.join(home_dir, hourly_diff_station_dir)):
    if file[-3:] == "csv":
        s_id = int(file.split("_")[1])
        hourly_diff_station_ids.append(s_id)
####

In [ ]:
####
maxlag = 24   # maxlag values: 3, 24, 72, 120

# Output folder
out_dir = home_dir + f'01_data/processed/csv/ARDL/hourly/lag{maxlag}/'

# Stations already processed
finished_stations = []
for file in os.listdir(out_dir):
    if file[-3:] == "csv":
        finished_stations.append(int(file.split('_')[1]))


# List of all stations ids
station_lst = list(set(hourly_df.station_id.unique()))

# Weather variables
predictor_lst = ['t2m', 'tp', 'wind_speed']

for s_id in tqdm(station_lst):
    # Skip processed stations
    if s_id in finished_stations:
        continue
    else:
        try:
            if s_id in hourly_diff_station_ids: # there are timeseries in the station df that is not stationary
                # First check the differenced time series data for this station
                # Note: here 't2m' is common but other variables may come up too so we need to account for that
                # Read the differenced time series
                station_diff_df = pd.read_csv(home_dir+f"01_data/processed/csv/diff/hourly_diff1/station_{s_id}_hourly_diff1.csv", index_col=0)
                # Get the original df of the station too
                station_df = hourly_df[hourly_df['station_id'] == s_id].sort_values(by='datetime')

                # Find columns to that needs to shift
                diffed_cols = [item.split("_diff")[0] for item in station_diff_df.columns.values[1:]]
                cols_to_shift = [x for x in predictor_lst if x not in diffed_cols]

                # Find how many positions to shift (1 or 2)
                shift_pos = station_df.shape[0] - station_diff_df.shape[0]
                
                # Shift the cols that were not differenced
                for col in cols_to_shift:
                    station_diff_df[f"{col}_shift{shift_pos}"] = station_df[col][shift_pos:].values
                
                # Now that the dataframe is ready, for ARDL
                # ARDL select order
                # Select correct lag order

                sel_res = ardl_select_order(
                                endog = station_diff_df.filter(regex='^pct_blackout').squeeze(), # engoenous var: hourly percent blackout
                                maxlag = maxlag,     # The maximum lag to consider for the endogenous variable.
                                exog = station_diff_df.iloc[:, 1:], # all columns except for the first col (pct blackout)
                                maxorder = maxlag,  # a common max lag length for all exog variables
                                trend = 'ct',       # Constant and time trend
                                causal = True,     # exclude lag 0 of exog variables
                                ic = 'bic',         # string
                                glob = True         # consider all possible submodels of the largest model or only if smaller order lags must be included if larger order lags are
                                )

                res = sel_res.model.fit() # Fit ARDL model
                res_dict = extract_ardl_results(res, s_id, sel_res)
                res_df = pd.DataFrame([res_dict])
                res_df.to_csv(out_dir + f'station_{s_id}_ardl_lag{maxlag}.csv')

            else:
                station_df = hourly_df[hourly_df['station_id'] == s_id].sort_values(by='datetime')
                # ARDL select order
                # Select correct lag order

                sel_res = ardl_select_order(
                                endog = station_df.filter(regex='^pct_blackout').squeeze(), # engoenous var: hourly percent blackout
                                maxlag = maxlag,     # The maximum lag to consider for the endogenous variable.
                                exog = station_df.iloc[:, 3:], # all columns except for station_id, datetime, pct blackout)
                                maxorder = maxlag,  # a common max lag length for all exog variables
                                trend = 'ct',       # Constant and time trend
                                causal = True,     # exclude lag 0 of exog variables
                                ic = 'bic',         # string
                                glob = True         # consider all possible submodels of the largest model or only if smaller order lags must be included if larger order lags are
                                )

                res = sel_res.model.fit() # Fit ARDL model
                res_dict = extract_ardl_results(res, s_id, sel_res)
                res_df = pd.DataFrame([res_dict])
                res_df.to_csv(out_dir + f'station_{s_id}_ardl_lag{maxlag}.csv')
        except ValueError:
            print(f"Visit station {s_id} later")

  0%|          | 0/519 [00:00<?, ?it/s]

## Daily

Define function that runs ARDL on target and predictor time series.

In [2]:
def run_ARDL_findlag_daily(maxlag, target, predictor_lst, station_lst, ic, daily_df):
    # Initiate an empty dictionary
    res_dic = {}

    # Iterate through all stations
    for s_id in tqdm(station_lst):
            try:
                # Subset station data
                station_df = daily_df[daily_df['station_id'] == s_id].sort_values(by='date')
                # Ignore stations with less than 90 days/ 3 months worth of data
                if len(station_df) <= 90:
                     continue
                else:
                    # Find optimum lag and store station id with optimum lag in dictionary
                    sel_res = ardl_select_order(
                        station_df[target],
                        exog=station_df[predictors_lst],
                        maxlag=maxlag,
                        ic= ic,# string
                        maxorder=maxlag,
                        causal=True,
                        trend='ct'
                        )
                    # optimum lag values for endogenous variables
                    endo_res_lst = list(sel_res.bic.values[0][1].values())
                    # insert the optimum lag value for exdogenous variable
                    endo_res_lst.insert(0, sel_res.bic.values[0][0])
                    # store station id and optimum lag values in dictionary
                    res_dic[s_id] = endo_res_lst # key: station id, value:[pct_blackout_lag, t2m_lag, wind_speed_lag, tp_lag]
            except ValueError:
                 print('error')
    # Save results in dataframe             
    res_df = pd.DataFrame.from_dict(res_dic, orient='index')

    # Rename columns using meaningful names
    res_final_df = res_df.reset_index().rename(columns={**{'index': 'station_id', 0: 'blackout_minutes_lag'},
                                                        **{i+1: val+'_lag' for i, val in enumerate(predictors_lst)}})
    return res_final_df

In [ ]:
# Set home directory
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/'
# Load station daily data
daily_df = pd.read_csv(home_dir + "01_data/processed/csv/daily_519station_13weather.csv", index_col=0)

# Loop through different maxlag values
for maxlag in [5, 10, 15, 30, 45, 60]: # unit: days
    print(f'I am working on maxlag={maxlag} now.')
    # Set target variable
    target = 'daily_blackout_minutes'
    # Set list of predictor variables
    predictors_lst = [
                    't2m_max',
                    'wind_speed_max',
                    'tp_max']
    # list of unique station ids
    station_lst = list(set(daily_df.station_id.unique()))
    # Information criterion
    ic = 'bic'
    # Run ARDL find lag
    result_df = run_ARDL_findlag_daily(maxlag, target, predictor_lst, station_lst, ic, daily_df)
    # Save output dataframe to csv
    result_df.to_csv(home_dir + f"01_data/processed/csv/ARDL_daily_optimum_{ic}_max{maxlag}.csv")